---
title: "Workshop 7: Using LLMs to Extract Information from Job Descriptions"
author-meta: "Emmanuel Dagba"
---


# Introduction

For this week's workshop, you will analyze job posting data using Large Language Models (LLMs). You'll extract structured information from job descriptions and create visualizations to understand patterns in the data science job market.

# Setup

First, make sure this document is in your `graph_courses_python` folder, where your virtual environment is activated and you have the required packages installed.

Run the following code to load the necessary packages:


In [ ]:
from openai import OpenAI
import pandas as pd
import numpy as np
import plotly.express as px
from local_settings import OPENAI_KEY # Assumes you have a local_settings.py file in your folder with your OpenAI key  
# Initialize the OpenAI client
client = OpenAI(api_key=OPENAI_KEY)

# Define an `llm_chat` Function

Before we start our analysis, let's create a helper function to simplify interactions with the LLM.

Test the function below by asking the question "What is Python (the language) named after?"


In [ ]:
def llm_chat(message):
    response = client.chat.completions.create(
        model="gpt-4o-mini", messages=[{"role": "user", "content": message}]
    )
    return response.choices[0].message.content

In [ ]:
# Test the function here
llm_chat("What is Python (the language) named after?")

# Get the Data

The dataset to be used contains job postings scraped from the website Glassdoor. You can see the scraping project [here](https://github.com/picklesueat/data_jobs_data).

Download it from the course website and place it in an appropriate location in your folder.

Load the data into a pandas DataFrame:


In [ ]:
# Load the data into a pandas DataFrame
# Your code here
jobs = pd.read_csv("data/glassdoor_jobs_sample.csv")
jobs

In [ ]:
jobs.info()

# Task 1: Extract Years of Experience Required

## Part A: Create and Test LLM Function

We will use the LLM to extract the minimum years of experience required from job descriptions.

We have written the prompt for you. Vectorize the function, then test it on the first few rows of the dataset.


In [ ]:
def yrs_exp(description):
    prompt = f"""
    Extract the minimum years of experience required from this job description. 
    Return ONLY a number. If a range is given, return the lower number.
    If no years of experience are explicitly mentioned, return 'NA'.
    Here's the job description:
    {description}
    """
    return llm_chat(prompt)

# Vectorize the function and test it on the first few rows
# Your code here
yrs_exp_vec = np.vectorize(yrs_exp)

In [ ]:
jobs_subset_test = jobs.head()
jobs_subset_test["years_experience"] = yrs_exp_vec(jobs_subset_test["job_description"])
jobs_subset_test

In [ ]:
jobs_subset_test.to_csv("outputs/jobs.csv")

Write the data to a CSV file and check if the results make sense by comparing them to the original job descriptions.

## Part B: Process Full Dataset

Now that we've confirmed the function works well, let's apply it to the full dataset. Note that this may take about 2 to 3 minutes to run. (For heavier workflows, we recommend you look into 'parallel processing'.)


In [ ]:
# Apply the vectorized function to the full dataset
# Your code here
jobs["years_experience"] = yrs_exp_vec(jobs["job_description"])
jobs

## Part C: Convert to Numeric

The `years_required` column is currently a string. Convert it to a numeric type using the `pd.to_numeric` function, with the `errors='coerce'` argument so that any non-numeric values are converted to `NaN`.


In [ ]:
jobs.info()
# Convert 'years_required' to numeric
# Your code here
jobs["years_experience"] = pd.to_numeric(jobs["years_experience"] , errors ="coerce")
jobs

## Part D: Create Visualization

Create a visualization comparing years of experience required to the midpoint salary estimate. You'll need to:

- Create a scatter plot using Plotly Express.
- Use the `midpoint_salary_estimate` column for salary and `years_required` for the x-axis.


In [ ]:
# Create a scatter plot
# Your code here
px.scatter(
    jobs, 
    x = "years_experience", 
    y = "midpoint_salary_estimate", 
    title = "Salary by years of experience required",
    labels = {"years_experience" : "Years of Experience Required", "midpoint_salary_estimate": "Salary"},
)

Describe any relationships you see in the plot.

The plot does not show a clear pattern but we can see that the years of experience required does not really have a strong correlation with the salary, however majority of the roles do not require years of experience greater than 5 years and the salary does not exceed 100k for majority of the roles except for a few outliers.

# Task 2: Extract Programming Language Requirements

In this task, we will ask the LLM to extract the programming languages mentioned in the job descriptions.

## Part A: Create and Test LLM Function

Now, create a function that asks the model about the programming languages mentioned in the job description. Specifically, it should return one of four categories regarding the languages mentioned: `"R"`, `"Python"`, `"both"`, or `"neither"`. This time, you'll need to craft the prompt yourself.

Test your function on a few rows and refine your prompt until you get reliable results. (You can write the output to a CSV file to more easily compare the results to the original job descriptions.)


In [ ]:
def lang_req(description):
    # Craft your prompt here
    prompt = f"""
   Consider the job description:
   Extract the programming languages following the guide:
   return "R" if R alone is mentioned,
   return "Python" if Python alone is mentioned,
   return "both" if both Python and R are mentioned,
   return "neither" if neither Python or R are mentioned
    Do not include explanations keep it structured
    Here's the job description:
     {description}
    """
    return llm_chat(prompt)

# Vectorize the function and test it on the first few rows
# Your code here
lang_req_vec = np.vectorize(lang_req)

In [ ]:
jobs_sample = jobs.head()

In [ ]:
jobs_sample ["reguired_language"] = lang_req_vec(jobs_sample["job_description"])

jobs_sample

## Part B: Process Full Dataset

Once you're satisfied with your function's performance, apply it to the full dataset:


In [ ]:
# Apply the function to the full dataset
# Your code here

jobs["reguired_language"] = lang_req_vec(jobs["job_description"])

jobs

## Part C: Create Visualization

First, count the number of jobs that require each of the four categories using the `value_counts` method. 


In [ ]:
# Count the occurrences of each category
# Your code here
jobs["reguired_language"].value_counts()

Create a box plot comparing salary distributions across the different programming language requirement categories:


In [ ]:
# Create a box plot using Plotly Express
# Your code here

px.box(jobs, x="reguired_language", y="midpoint_salary_estimate")

Write a few sentences describing any patterns you see in the plot. (Note that this is a very small sample of jobs, so don't read too much into the results.)

It is observed that roles with a combination of Python and R have higher salaries, however, roles requiring Python has more salary variations while the few salary outliers are found in roles which do not require any programming language.

# Optional Challenge: Most Common Technical Skills

Use an LLM function to extract the most common technical skills mentioned in job descriptions, then create a visualization to highlight these skills.

You will need to design your own approach to extract and standardize technical skills, being explicit in your prompt about what constitutes a technical skill.

There's no single correct way to classify and standardize skills—document your choices and reasoning as you develop your solution.


In [ ]:
def tech_skill(job):
    # Craft your prompt here
    prompt = f"""
   Consider the job_title:
   Generate required technical skill for each industry
     {job}
    """
    return llm_chat(prompt)

tech_skill_vec = np.vectorize (tech_skill)

In [ ]:
jobs["technical_skill"] = tech_skill_vec(jobs["industry"])

In [ ]:
jobs_subset_test["technical_skill"] = tech_skill_vec(jobs["job_title"])
jobs_subset_test